# 서울시 지하철 시간대별 유동인구 분석 (히트맵 + 프롬프트 출력 포함)

이 노트북은 시간대별 승하차 데이터를 기반으로 서울시 주요 혼잡 구간을 시각화하고,
출근/퇴근 시간대의 상위 10개 혼잡역을 히트맵 및 프롬프트로 출력합니다.

In [ ]:
import pandas as pd
import folium
from folium.plugins import HeatMap, MarkerCluster
from folium.features import DivIcon

# 데이터 로딩
df_time = pd.read_csv('seoul_subway_by_time.csv', encoding='cp949')
df_xy = pd.read_csv('seoul_subway_xy.csv')

In [ ]:
# 전처리 및 병합
df_time = df_time[['지하철역', '08시-09시 승차인원', '08시-09시 하차인원', '18시-19시 승차인원', '18시-19시 하차인원']].copy()
df_time['지하철역'] = df_time['지하철역'].str.strip()
df_grouped = df_time.groupby('지하철역').sum().reset_index()
df_xy['역한글명칭'] = df_xy['역한글명칭'].str.strip()
df_coords = df_xy.groupby('역한글명칭')[['환승역X좌표', '환승역Y좌표']].mean().reset_index()
df = pd.merge(df_grouped, df_coords, left_on='지하철역', right_on='역한글명칭', how='inner')
df['지도용X'] = df['환승역X좌표']
df['지도용Y'] = df['환승역Y좌표']

In [ ]:
def draw_heatmap(df, value_col, color, map_filename):
    m = folium.Map(location=[37.5665, 126.9780], zoom_start=11, tiles='CartoDB positron')
    cluster = MarkerCluster().add_to(m)
    heat_data = df[['지도용Y', '지도용X', value_col]].dropna().values.tolist()
    HeatMap(heat_data, radius=15).add_to(m)
    top10 = df.sort_values(by=value_col, ascending=False).head(10).reset_index(drop=True)
    for i, row in top10.iterrows():
        popup = f"{row['지하철역']}<br>{value_col}: {int(row[value_col]):,}명"
        folium.CircleMarker(
            location=[row['지도용Y'], row['지도용X']],
            radius=9,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.9,
            popup=popup
        ).add_to(cluster)
        folium.map.Marker(
            [row['지도용Y'], row['지도용X']],
            icon=DivIcon(
                icon_size=(20, 20),
                icon_anchor=(5, -5),
                html=f'<div style="font-size:12pt; color:white; background:{color}; border-radius:50%; width:22px; height:22px; text-align:center;">{i+1}</div>'
            )
        ).add_to(m)
    m.save(map_filename)
    print(f"\n📍 {value_col} 기준 상위 10개 역 (일평균):")
    print((top10[["지하철역", value_col]] / 31).round().astype(int).to_string(index=False))

In [ ]:
# 히트맵 생성 실행
draw_heatmap(df, '08시-09시 승차인원', 'blue', 'morning_boarding.html')
draw_heatmap(df, '08시-09시 하차인원', 'green', 'morning_alighting.html')
draw_heatmap(df, '18시-19시 승차인원', 'blue', 'evening_boarding.html')
draw_heatmap(df, '18시-19시 하차인원', 'green', 'evening_alighting.html')